# Load 'users_001.csv' into DataFrame

In [0]:
df = spark.read.csv(
    path="/Volumes/quickstart_catalog/quickstart_schema/sandbox/dataset/user_dataset/users_001.csv",
    header=True,
    inferSchema=True,
)
df.display()
 

## Tansaction 01 - Write data as a Delta Format

In [0]:
df.write.format("delta").save("/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta")

## Tansaction 02 - Overwrite 
here it will not overwrite the data by deleting old data instead it will add the new data without affecting old data


In [0]:
df.write.format("delta").save("/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta_new")

In [0]:
#here the output_delta_new consists of both country i,e United States and India

spark.read.load("/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta_new").display()

In [0]:
#here the output_delta_new consists of country i,e India
from pyspark.sql.functions import col
df.filter(col("country")=="India").write.save("/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta",mode="overwrite")


## Maintains Versioning (Advanatge)

In [0]:
spark.read.option("versionAsof", 0).load("/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta_new").display()

In [0]:
spark.read.load(
    format="text",
    path="/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta_new/_delta_log/00000000000000000001.json",
).display()

## Approach 02 - to see the latest transaction 

In [0]:
from delta.tables import DeltaTable
 
delta_table = DeltaTable.forPath(
    spark, "/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta_new/"
)
delta_table.history().display()

# Transaction 03 - Update is allowed

In [0]:
df_delta = spark.read.load(
    "/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta_new/"
)
df_delta.createOrReplaceTempView("user_vw")

In [0]:
%sql
 
UPDATE user_vw
SET country='Bharat'
where country='India'